## Computation of EWBI and wellbeing sub-indicators

In [18]:
import json
import numpy as np
import pandas as pd

In [19]:
df = pd.read_csv('data/df_final_EWBI.csv')
df

,year,country,decile,primary_index,value,database,quintile
0,2004.0,AT,1.0,EL-SILC-1,0.0,EU-SILC,NaN
1,2004.0,AT,2.0,EL-SILC-1,0.0,EU-SILC,NaN
2,2004.0,AT,3.0,EL-SILC-1,0.0,EU-SILC,NaN
3,2004.0,AT,4.0,EL-SILC-1,0.0,EU-SILC,NaN
4,2004.0,AT,5.0,EL-SILC-1,0.0,EU-SILC,NaN
...,...,...,...,...,...,...,...
176783,2015.0,SE,NaN,AB-EHIS-2,0.0,EHIS,2.0
176784,2015.0,SE,NaN,AB-EHIS-2,0.0,EHIS,3.0
176785,2015.0,SE,NaN,AB-EHIS-2,0.0,EHIS,4.0
176786,2015.0,SE,NaN,AB-EHIS-2,0.0,EHIS,5.0


## Preprocessing
### Data cleaning

In [3]:
df = df.drop(columns=['database'])
df['year'] = df['year'].astype(int)
df = df[~df['year'].isin([-1, 2118])]
df.year.unique()

array([2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 1983, 1984,
       1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003])

### Splitting quintiles into 2 deciles

In [ ]:
def process_quantiles(df):
    """
    Port data in quintiles to deciles by assigning duplicating each row with quintile 
    and assigning it to the two corresponding deciles.
    """
    print("Initial length:", len(df))
    df = df[df.quintile != -1]
    print("After removing quintile = -1:", len(df))
    df = df[df.decile != 99]
    print("After removing decile = 99:", len(df))
    quintile_rows = df[df.quintile.notna()].copy()
    quintile_rows['decile'] = quintile_rows['quintile'] * 2
    print("Number of rows with quintile:", len(quintile_rows))
    quintile_rows_duplicated = quintile_rows.copy()
    quintile_rows_duplicated['decile'] = quintile_rows_duplicated['quintile'] * 2 - 1
    df = pd.concat([df[df.quintile.isna()], quintile_rows, quintile_rows_duplicated], ignore_index=True)
    print("Final length:", len(df))
    df['decile'] = df['decile'].astype(int)
    df = df.drop(columns=['quintile'])
    return df

df = process_quantiles(df)

Initial length: 176711
After removing quintile = -1: 176447
After removing decile = 99: 172587
Number of rows with quintile: 3025
Final length: 175612


### Fill NaNs
We take a rather crude approach filling all years for all indicators even in some indicators have data only for a few years.

In [ ]:
wide = df.pivot_table(values='value', index=['primary_index', 'decile', 'country'], columns='year')
wide

year                          2004  2005  2006  2007  2008  2009      2010  \
primary_index decile country                                                 
AB-EHIS-1     1      AT        NaN   NaN   NaN   NaN   NaN   NaN       NaN   
                     BE        NaN   NaN   NaN   NaN   NaN   NaN       NaN   
                     BG        NaN   NaN   NaN   NaN   NaN   NaN       NaN   
                     CY        NaN   NaN   NaN   NaN   NaN   NaN       NaN   
                     CZ        NaN   NaN   NaN   NaN   NaN   NaN       NaN   
...                            ...   ...   ...   ...   ...   ...       ...   
TT-HBS-2      10     RO        NaN   NaN   NaN   NaN   NaN   NaN  0.208648   
                     SE        NaN   NaN   NaN   NaN   NaN   NaN  0.366487   
                     SI        NaN   NaN   NaN   NaN   NaN   NaN  0.227785   
                     SK        NaN   NaN   NaN   NaN   NaN   NaN  0.212835   
                     UK        NaN   NaN   NaN   NaN   NaN   NaN  0.294818   

year                          2011  2012      2013      2014      2015  2016  \
primary_index decile country                                                   
AB-EHIS-1     1      AT        NaN   NaN  0.145907  0.130969  0.145797   NaN   
                     BE        NaN   NaN  0.092452       NaN       NaN   NaN   
                     BG        NaN   NaN       NaN  0.153553  0.000000   NaN   
                     CY        NaN   NaN       NaN  0.144518       NaN   NaN   
                     CZ        NaN   NaN       NaN  0.126716  0.216426   NaN   
...                            ...   ...       ...       ...       ...   ...   
TT-HBS-2      10     RO        NaN   NaN       NaN       NaN  0.218466   NaN   
                     SE        NaN   NaN       NaN       NaN  0.361924   NaN   
                     SI        NaN   NaN       NaN       NaN  0.202895   NaN   
                     SK        NaN   NaN       NaN       NaN  0.259485   NaN   
                     UK        NaN   NaN       NaN       NaN       NaN   NaN   

year                          2017  2018  2019      2020  2021  2022  2023  
primary_index decile country                                                
AB-EHIS-1     1      AT        NaN   NaN   NaN       NaN   NaN   NaN   NaN  
                     BE        NaN   NaN   NaN       NaN   NaN   NaN   NaN  
                     BG        NaN   NaN   NaN       NaN   NaN   NaN   NaN  
                     CY        NaN   NaN   NaN       NaN   NaN   NaN   NaN  
                     CZ        NaN   NaN   NaN       NaN   NaN   NaN   NaN  
...                            ...   ...   ...       ...   ...   ...   ...  
TT-HBS-2      10     RO        NaN   NaN   NaN  0.162345   NaN   NaN   NaN  
                     SE        NaN   NaN   NaN       NaN   NaN   NaN   NaN  
                     SI        NaN   NaN   NaN  0.249146   NaN   NaN   NaN  
                     SK        NaN   NaN   NaN  0.276132   NaN   NaN   NaN  
                     UK        NaN   NaN   NaN       NaN   NaN   NaN   NaN  

[16064 rows x 20 columns]

In [17]:
filled = wide.ffill(axis=1).bfill(axis=1)
filled

year                              2004      2005      2006      2007  \
primary_index decile country                                           
AB-EHIS-1     1      AT       0.145907  0.145907  0.145907  0.145907   
                     BE       0.092452  0.092452  0.092452  0.092452   
                     BG       0.153553  0.153553  0.153553  0.153553   
                     CY       0.144518  0.144518  0.144518  0.144518   
                     CZ       0.126716  0.126716  0.126716  0.126716   
...                                ...       ...       ...       ...   
TT-HBS-2      10     RO       0.208648  0.208648  0.208648  0.208648   
                     SE       0.366487  0.366487  0.366487  0.366487   
                     SI       0.227785  0.227785  0.227785  0.227785   
                     SK       0.212835  0.212835  0.212835  0.212835   
                     UK       0.294818  0.294818  0.294818  0.294818   

year                              2008      2009      2010      2011  \
primary_index decile country                                           
AB-EHIS-1     1      AT       0.145907  0.145907  0.145907  0.145907   
                     BE       0.092452  0.092452  0.092452  0.092452   
                     BG       0.153553  0.153553  0.153553  0.153553   
                     CY       0.144518  0.144518  0.144518  0.144518   
                     CZ       0.126716  0.126716  0.126716  0.126716   
...                                ...       ...       ...       ...   
TT-HBS-2      10     RO       0.208648  0.208648  0.208648  0.208648   
                     SE       0.366487  0.366487  0.366487  0.366487   
                     SI       0.227785  0.227785  0.227785  0.227785   
                     SK       0.212835  0.212835  0.212835  0.212835   
                     UK       0.294818  0.294818  0.294818  0.294818   

year                              2012      2013      2014      2015  \
primary_index decile country                                           
AB-EHIS-1     1      AT       0.145907  0.145907  0.130969  0.145797   
                     BE       0.092452  0.092452  0.092452  0.092452   
                     BG       0.153553  0.153553  0.153553  0.000000   
                     CY       0.144518  0.144518  0.144518  0.144518   
                     CZ       0.126716  0.126716  0.126716  0.216426   
...                                ...       ...       ...       ...   
TT-HBS-2      10     RO       0.208648  0.208648  0.208648  0.218466   
                     SE       0.366487  0.366487  0.366487  0.361924   
                     SI       0.227785  0.227785  0.227785  0.202895   
                     SK       0.212835  0.212835  0.212835  0.259485   
                     UK       0.294818  0.294818  0.294818  0.294818   

year                              2016      2017      2018      2019  \
primary_index decile country                                           
AB-EHIS-1     1      AT       0.145797  0.145797  0.145797  0.145797   
                     BE       0.092452  0.092452  0.092452  0.092452   
                     BG       0.000000  0.000000  0.000000  0.000000   
                     CY       0.144518  0.144518  0.144518  0.144518   
                     CZ       0.216426  0.216426  0.216426  0.216426   
...                                ...       ...       ...       ...   
TT-HBS-2      10     RO       0.218466  0.218466  0.218466  0.218466   
                     SE       0.361924  0.361924  0.361924  0.361924   
                     SI       0.202895  0.202895  0.202895  0.202895   
                     SK       0.259485  0.259485  0.259485  0.259485   
                     UK       0.294818  0.294818  0.294818  0.294818   

year                              2020      2021      2022      2023  
primary_index decile country                                          
AB-EHIS-1     1      AT       0.145797  0.145797  0.145797  0.145797  
                     BE       0.092452 

### Normalising

In [29]:
# The normalisation is intra-decile and intra-indicator so we separate using groupby
res = []
for (ind, decile), grouped in filled.groupby(['primary_index', 'decile']):
    data = grouped.copy()

    # normalize the data over countries, so that the best-performing coutry has value 1 and the worst 0
    # values are negative in the sense that the best-performing country is the one with the lowest initial value and vice-versa
    norm = 1 - (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))

    # replace 0 values with 0.001
    norm = norm.replace(0, 0.001)
    res.append(norm)

preprocessed = pd.concat(res)
preprocessed

year                              2004      2005      2006      2007  \
primary_index decile country                                           
AB-EHIS-1     1      AT       0.189613  0.189613  0.189613  0.189613   
                     BE       0.636030  0.636030  0.636030  0.636030   
                     BG       0.125760  0.125760  0.125760  0.125760   
                     CY       0.201209  0.201209  0.201209  0.201209   
                     CZ       0.349882  0.349882  0.349882  0.349882   
...                                ...       ...       ...       ...   
TT-HBS-2      10     RO       0.712444  0.712444  0.712444  0.712444   
                     SE       0.113006  0.113006  0.113006  0.113006   
                     SI       0.639766  0.639766  0.639766  0.639766   
                     SK       0.696545  0.696545  0.696545  0.696545   
                     UK       0.385191  0.385191  0.385191  0.385191   

year                              2008      2009      2010      2011  \
primary_index decile country                                           
AB-EHIS-1     1      AT       0.189613  0.189613  0.189613  0.189613   
                     BE       0.636030  0.636030  0.636030  0.636030   
                     BG       0.125760  0.125760  0.125760  0.125760   
                     CY       0.201209  0.201209  0.201209  0.201209   
                     CZ       0.349882  0.349882  0.349882  0.349882   
...                                ...       ...       ...       ...   
TT-HBS-2      10     RO       0.712444  0.712444  0.712444  0.712444   
                     SE       0.113006  0.113006  0.113006  0.113006   
                     SI       0.639766  0.639766  0.639766  0.639766   
                     SK       0.696545  0.696545  0.696545  0.696545   
                     UK       0.385191  0.385191  0.385191  0.385191   

year                              2012      2013      2014      2015  \
primary_index decile country                                           
AB-EHIS-1     1      AT       0.189613  0.189613  0.314359  0.326344   
                     BE       0.636030  0.636030  0.636030  0.572825   
                     BG       0.125760  0.125760  0.125760  1.000000   
                     CY       0.201209  0.201209  0.201209  0.332252   
                     CZ       0.349882  0.349882  0.349882  0.001000   
...                                ...       ...       ...       ...   
TT-HBS-2      10     RO       0.712444  0.712444  0.712444  0.689601   
                     SE       0.113006  0.113006  0.113006  0.133124   
                     SI       0.639766  0.639766  0.639766  0.750003   
                     SK       0.696545  0.696545  0.696545  0.530487   
                     UK       0.385191  0.385191  0.385191  0.393431   

year                              2016      2017      2018      2019  \
primary_index decile country                                           
AB-EHIS-1     1      AT       0.326344  0.326344  0.326344  0.326344   
                     BE       0.572825  0.572825  0.572825  0.572825   
                     BG       1.000000  1.000000  1.000000  1.000000   
                     CY       0.332252  0.332252  0.332252  0.332252   
                     CZ       0.001000  0.001000  0.001000  0.001000   
...                                ...       ...       ...       ...   
TT-HBS-2      10     RO       0.689601  0.689601  0.689601  0.689601   
                     SE       0.133124  0.133124  0.133124  0.133124   
                     SI       0.750003  0.750003  0.750003  0.750003   
                     SK       0.530487  0.530487  0.530487  0.530487   
                     UK       0.393431  0.393431  0.393431  0.393431   

year                              2020      2021      2022      2023  
primary_index decile country                                          
AB-EHIS-1     1      AT       0.326344  0.326344  0.326344  0.326344  
                     BE       0.572825 

## Performing the computation

In [88]:
def weighted_geometric_mean(data: list[tuple[pd.Series | pd.DataFrame, float]]):
    # computed in log space for numerical stability
    wsum = 0
    logsum = 0
    for values, weight in data:
        logsum += weight * np.log(values)
        wsum += weight
    
    return np.exp(logsum / wsum)

In [ ]:
with open('ewbi_indicators.json') as f:
    weights = json.load(f)['EWBI']

In [59]:
all_codes = set()
all_secondaries = set()
for prio in weights:
    print(prio['name'])
    for component in prio['components']:
        print('\t', component['name'])
        print(f'\t\t {component['name']}: {component['weight']}')
        all_secondaries.add(component['name'])
        for ind in component['indicators']:
            print(f'\t\t\t {ind['code']}: {ind['weight']}')
            all_codes.add(ind['code'])

Agriculture and Food
	 Nutrition need
		 Nutrition need: 2/3
			 AN-SILC-1: 2
			 AF-EHIS-1: 2
	 Nutrition expense
		 Nutrition expense: 1/3
			 AE-HBS-1: 1
			 AE-HBS-2: 1
			 AE-EHIS-1: 2
Energy and Housing
	 Housing quality
		 Housing quality: 1/3
			 HQ-SILC-1: 2
			 HQ-SILC-2: 2
	 Energy
		 Energy: 1/3
			 HE-SILC-1: 2
			 HE-SILC-2: 1
	 Housing expense
		 Housing expense: 1/3
			 HH-SILC-1: 1
			 HH-HBS-1: 1
			 HH-HBS-2: 1
			 HH-HBS-3: 1
			 HH-HBS-4: 1
Equality
	 Life satisfaction
		 Life satisfaction: 1/4
			 EL-SILC-1: 2
			 EL-EHIS-1: 2
	 Security
		 Security: 1/4
			 ES-SILC-1: 2
			 ES-SILC-2: 2
	 Community
		 Community: 1/4
			 EC-SILC-1: 2
			 EC-SILC-2: 2
			 EC-HBS-1: 1
			 EC-HBS-2: 1
			 EC-EHIS-1: 2
	 Digital Skills
		 Digital Skills: 1/4
			 ED-EHIS-1: 2
Health and Animal Welfare
	 Health condition and impact
		 Health condition and impact: 1/2
			 AH-SILC-1: 2
			 AA-SILC-1: 2
			 AA-SILC-2: 2
			 AH-SILC-2: 2
			 AH-EHIS-1: 2
	 Health cost and medical care
		 He

In [89]:
print(all_codes.difference(preprocessed.index.get_level_values('primary_index')))
print(preprocessed.index.get_level_values('primary_index').difference(all_codes))

{'AF-EHIS-1', 'EC-SILC-1', 'IL-SILC-3', 'AA-SILC-2', 'IS-LFS-1', 'AA-SILC-1', 'EC-HBS-1', 'TS-HBS-1', 'TS-HBS-2', 'EC-HBS-2'}
Index(['AC-SILC-2', 'AH-EHIS-2', 'AH-SILC-3', 'AH-SILC-4', 'AN-EHIS-1',
       'IS-SILC-3'],
      dtype='object', name='primary_index')


In [60]:
secondary = {}
for prio in weights:
    for component in prio['components']:
        factors = []
        for ind in component['indicators']:
            code = ind['code']
            weight = ind['weight']
            if code in preprocessed.index:
                factors.append((preprocessed.loc[code], weight))

        secondary[prio['name'], component['name']] = weighted_geometric_mean(factors)

secondary = pd.concat(secondary, names=['eu_priority', 'secondary_index'])
secondary

year                                                                  2004  \
eu_priority                       secondary_index decile country             
Agriculture and Food              Nutrition need  1      AT       0.223442   
                                                         BE       0.128649   
                                                         BG       1.000000   
                                                         CH       0.005751   
                                                         CY       0.174586   
...                                                                    ...   
Sustainable Transport and Tourism Tourism         10     RS       0.527906   
                                                         SE       0.938578   
                                                         SI       0.794275   
                                                         SK       0.292747   
                                                         UK       0.917401   

year                                                                  2005  \
eu_priority                       secondary_index decile country             
Agriculture and Food              Nutrition need  1      AT       0.239554   
                                                         BE       0.104725   
                                                         BG       1.000000   
                                                         CH       0.001000   
                                                         CY       0.169811   
...                                                                    ...   
Sustainable Transport and Tourism Tourism         10     RS       0.526505   
                                                         SE       0.945121   
                                                         SI       0.792166   
                                                         SK       0.291970   
                                                         UK       0.914966   

year                                                                  2006  \
eu_priority                       secondary_index decile country             
Agriculture and Food              Nutrition need  1      AT       0.218062   
                                                         BE       0.125696   
                                                         BG       1.000000   
                                                         CH       0.010988   
                                                         CY       0.141327   
...                                                                    ...   
Sustainable Transport and Tourism Tourism         10     RS       0.530147   
                                                         SE       0.944203   
                                                         SI       0.800670   
                                                         SK       0.373994   
                                                         UK       0.932660   

year                                                                  2007  \
eu_priority                       secondary_index decile country             
Agriculture and Food              Nutrition need  1      AT       0.280707   
                                                         BE       0.146690   
                                                         BG       1.000000   
                                                         CH       0.026747   
                                                         CY       0.272247   
...                                                                    ...   
Sustainable Transport and Tourism Tourism         10     RS       0.529300   
                                                         SE       0.986659   
                                                         SI       0.793513   
                                                         SK       0.503565   
                   

In [ ]:
secondary.to_csv('results/secondary_indicators.csv')

In [62]:
print(all_secondaries.difference(secondary.index.get_level_values('secondary_index')))
print(secondary.index.get_level_values('secondary_index').difference(all_secondaries))

set()
Index([], dtype='object', name='secondary_index')


In [82]:
priorities = {}
for prio in weights:
    pname = prio['name']
    factors = []
    for c in prio['components']:
        name = c['name']
        weight = c['weight']
        try:
          weight = float(weight)
        except ValueError:
          numerator, denominator = map(int, weight.split('/'))
          weight = float(numerator) / denominator

        factors.append((secondary.loc[pname, name], weight))

    priorities[pname] = weighted_geometric_mean(factors)

priorities = pd.concat(priorities, names=['eu_priority'])
priorities

/var/folders/fk/7hwfblcx5x39964xs_yhkqb00000gn/T/ipykernel_2391/4132978685.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  factors.append((secondary.loc[pname, name], weight))


year                                                  2004      2005  \
eu_priority                       decile country                       
Agriculture and Food              1      AT       0.314774  0.329730   
                                         BE       0.221916  0.193472   
                                         BG       0.646538  0.646538   
                                         CH            NaN       NaN   
                                         CY       0.215010  0.211072   
...                                                    ...       ...   
Sustainable Transport and Tourism 10     RO       0.317174  0.316893   
                                         SE       0.457500  0.458561   
                                         SI       0.738436  0.737782   
                                         SK       0.568990  0.568486   
                                         UK       0.674747  0.674150   

year                                                  2006      2007  \
eu_priority                       decile country                       
Agriculture and Food              1      AT       0.309701  0.366487   
                                         BE       0.218507  0.242206   
                                         BG       0.646538  0.646538   
                                         CH            NaN       NaN   
                                         CY       0.186753  0.289131   
...                                                    ...       ...   
Sustainable Transport and Tourism 10     RO       0.317622  0.317453   
                                         SE       0.458412  0.465182   
                                         SI       0.740413  0.738200   
                                         SK       0.617393  0.681751   
                                         UK       0.678468  0.677408   

year                                                  2008      2009  \
eu_priority                       decile country                       
Agriculture and Food              1      AT       0.399000  0.389173   
                                         BE       0.270809  0.266332   
                                         BG       0.646538  0.646538   
                                         CH            NaN       NaN   
                                         CY       0.212542  0.220507   
...                                                    ...       ...   
Sustainable Transport and Tourism 10     RO       0.156573  0.043508   
                                         SE       0.461672  0.462255   
                                         SI       0.713679  0.699740   
                                         SK       0.490853  0.509816   
                                         UK       0.651547  0.660570   

year                                                  2010      2011  \
eu_priority                       decile country                       
Agriculture and Food              1      AT       0.370249  0.347552   
                                         BE       0.284770  0.272254   
                                         BG       0.646538  0.646538   
                                         CH            NaN       NaN   
                                         CY       0.147052  0.185378   
...                                                    ...       ...   
Sustainable Transport and Tourism 10     RO       0.043508  0.043508   
                                         SE       0.462278  0.464160   
                                         SI       0.724754  0.724993   
                                         SK       0.555328  0.626597   
                                         UK       0.656375  0.651471   

year                                                  2012      2013  \
eu_priority                       decile country                       
Agriculture and Food              1      AT       0.315383  0.336159   
                                    

In [87]:
priorities.to_csv('results/eu_priorities.csv')

In [90]:
factors = [(priorities.loc[prio], 1) for prio in priorities.index.get_level_values('eu_priority')]
ewbi = weighted_geometric_mean(factors)

In [97]:
ewbi

year            2004  2005  2006  2007  2008  2009  2010  2011  2012  2013  \
decile country                                                               
1      AT        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       BE        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       BG        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       CH        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       CY        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...              ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
10     RS        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       SE        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       SI        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       SK        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       UK        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

year            2014  2015  2016  2017  2018  2019  2020  2021  2022  2023  
decile country                                                              
1      AT        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
       BE        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
       BG        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
       CH        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
       CY        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
...              ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  
10     RS        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
       SE        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
       SI        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
       SK        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
       UK        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[320 rows x 20 columns]

We have a NaN problem : we should be able to ignore NaNs

In [96]:
priorities.loc['Equality']

year            2004  2005  2006  2007  2008  2009  2010  2011  2012  2013  \
decile country                                                               
1      AT        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       BE        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       BG        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       CH        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       CY        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...              ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
10     RS        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       SE        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       SI        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       SK        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
       UK        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

year            2014  2015  2016  2017  2018  2019  2020      2021      2022  \
decile country                                                                 
1      AT        NaN   NaN   NaN   NaN   NaN   NaN   NaN  0.811056  0.792132   
       BE        NaN   NaN   NaN   NaN   NaN   NaN   NaN  0.685522  0.668878   
       BG        NaN   NaN   NaN   NaN   NaN   NaN   NaN  0.040486  0.041960   
       CH        NaN   NaN   NaN   NaN   NaN   NaN   NaN       NaN       NaN   
       CY        NaN   NaN   NaN   NaN   NaN   NaN   NaN  0.127183  0.268921   
...              ...   ...   ...   ...   ...   ...   ...       ...       ...   
10     RS        NaN   NaN   NaN   NaN   NaN   NaN   NaN       NaN       NaN   
       SE        NaN   NaN   NaN   NaN   NaN   NaN   NaN  0.409363  0.396577   
       SI        NaN   NaN   NaN   NaN   NaN   NaN   NaN  0.873898  0.811540   
       SK        NaN   NaN   NaN   NaN   NaN   NaN   NaN  0.646364  0.591410   
       UK        NaN   NaN   NaN   NaN   NaN   NaN   NaN  0.815224  0.779232   

year                2023  
decile country            
1      AT       0.768087  
       BE       0.687814  
       BG       0.039956  
       CH            NaN  
       CY       0.267067  
...                  ...  
10     RS            NaN  
       SE       0.378567  
       SI       0.821387  
       SK       0.626399  
       UK       0.821904  

[320 rows x 20 columns]